In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

In [2]:
import time
import csv
import numpy as np
import pandas as pd
import requests
import re

from IPython.core.display import HTML

In [4]:
def sign_in(email, password, PATH):
#     this function automates login process on linkedin, 
#     provide email and password as strings
    driver = webdriver.Chrome(PATH)
    time.sleep(5)
    # webdriver gets page
    driver.get("https://www.linkedin.com/")
    # pause for page to load
    time.sleep(3)
    # locate and send login email and password
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()
    return driver

In [173]:
def people_scrape(search_term, num_pages):
        
#                 function to automate search bar, search focus
#                 from your linkedin homepage, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.

    # activate search bar cursor with click
    driver.find_element_by_id("global-nav-search").click()
    # send keyboard entry for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(6)
    #w.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.reusable-search__entity-results-list')))
    element = driver.find_element_by_css_selector('ul.reusable-search__entity-results-list ')
    # scroll to element containing target(people_banner)  allowing ajax elements to load
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.implicitly_wait(5)
   # locate banner under search type results
    people_banner = driver.find_element_by_link_text(f"See all people results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',people_banner)
    # pause to allow page to load
    driver.implicitly_wait(6)
    # starting with pagination page 1                       
    page_number = 1
    # get current url for page
    current_page_url = driver.current_url

    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
        print("Processing page: " + str(page_number))
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    
    # iterate through results
        for l in links:
    # retrieve profile url
            title = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
            profile_path = (str(title.get_attribute("pathname")))
    # add to urls list
            url_list.append(profile_path)
    # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name = text[0]
            name_list.append(name)
    # retrieve location/add to list   
            loc = text[-1]
            loc_list.append(loc)
     # retrieve headline/add to list
            headline = text[-2]
            headline_list.append(headline)
    # the second element selected contains the 'Current:' job text 
            current_job = details[1]
    # removing the 'Current:' string from text
            current_job = current_job.get_attribute('innerText').split(':')[1]
    # add current job to list
            current_job_list.append(current_job)
        time.sleep(3)
    # navigate using pagination function
        goto_next_page()
        page_number+=1
    # print to verify page during processing
        print(f"attempting to navigate to search results page {page_number}")
        time.sleep(5)
   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = name_list
    df['url'] = url_list
    df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{search_term}.csv')
    # verify save
    print(f'{search_term}.csv saved')
    return df

In [124]:
def get_full_name():
    profile_header = driver.find_element_by_css_selector("ul.pv-top-card--list")
    # find name element from profile header portion
    header_details = profile_header.find_elements_by_tag_name("li")
    profile_name = header_details[0].get_attribute('innerText')
    return profile_name

def get_first_name():
    full_name = get_name()
    name = full_name.split(' ', 1)
    first_name = name[0]
    return first_name

In [123]:
#function to locate and interact with "next" button at bottom of search
def goto_next_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    driver.find_element_by_css_selector('div.artdeco-pagination')
    go_to_next = driver.find_element_by_css_selector('button[aria-label="Next"]')
    driver.execute_script('arguments[0].click();',go_to_next)

In [371]:
def get_experience():
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    time.sleep(5)
    # background container
    background = driver.find_element_by_css_selector(
        "div[id='oc-background-section']")
    #print("here" + str(background))
    driver.execute_script("arguments[0].scrollIntoView();", background)
    time.sleep(8)
    # try/except clause used to locate element, not all people list jobs
    exp = background.find_element_by_css_selector(
            "section[id='experience-section']")
    history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
    details = history[0]
    # try/except clause used to locate element to avoid error ceasing from running
    try:
        pos = details.find_element_by_tag_name(
                'h3').get_attribute('outerText')
    except NoSuchElementException:
        pos = 'nan'

    try:
        company = details.find_element_by_tag_name(
                'p.pv-entity__secondary-title').get_attribute('innerText')
        
    except NoSuchElementException:
        company = 'nan'
    try:
        location = details.find_element_by_css_selector(
                'h4.pv-entity__location').get_attribute('innerText')
        location = location.split('\n', 1)[1]
    except NoSuchElementException:
        location = 'nan'
        
    try:
        dates = details.find_element_by_css_selector(
                "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

    except NoSuchElementException:
            dates = 'nan'
    finally:
        job_list.append(pos)
        company_list.append(company)
        date_list.append(dates)
        loc_list.append(location)

    if (len(history)) >=2:
        details = history[1]
        try:
            pos = details.find_element_by_tag_name(
                    'h3').get_attribute('outerText')
        except NoSuchElementException:
            pos = 'nan'
     
        try:
            company = details.find_element_by_tag_name(
                    'p.pv-entity__secondary-title').get_attribute('innerText')

        except NoSuchElementException:
            company = 'nan'
        try:
            location = details.find_element_by_css_selector(
                    'h4.pv-entity__location').get_attribute('innerText')
            location = location.split('\n', 1)[1]
        except NoSuchElementException:
            location = 'nan'
                #print(location)
        try:
            dates = details.find_element_by_css_selector(
                    "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

        except NoSuchElementException:
                dates = 'nan'

        if (len(history)) < 2:
            pos = 'nan'
            company  ='nan'
            location  ='nan'
            dates = 'nan'

        job_list2.append(pos)
        company_list2.append(company)
        date_list2.append(dates)
        loc_list2.append(location)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    
    time.sleep(2)



In [372]:
def get_education():
    # wait for element to proceed
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    # background container
    background = driver.find_element_by_css_selector(
        "div[id='oc-background-section']")
    #print("here" + str(background))
    driver.execute_script("arguments[0].scrollIntoView();", background)
    time.sleep(8)
    # locate education section
    education = background.find_element_by_css_selector(
            "section[id='education-section']")
 

    try:
        
        schools = education.find_elements_by_css_selector(
                "a[data-control-name='background_details_school']")
        school = schools[0]
    except NoSuchElementException:
        school_name =  'nan'
    else:
        school_name = school.find_element_by_tag_name(
            'h3').get_attribute('innerText')
    school_list.append(school_name)

In [373]:
def get_email():
    # scroll to top of profile where email is located
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    contact_info = driver.find_element_by_css_selector(
        'a[data-control-name="contact_see_more"]')
     # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',contact_info)
    time.sleep(3)
    try:
        container =  driver.find_elements_by_css_selector(
            'div.pv-contact-info__ci-container')
    
    except NoSuchElementException:
        email = 'nan'

    else:
        try:
            email =  container[1].get_attribute('innerText')
     
        except IndexError:
            email = 'nan'
        email_list.append(email)
    close_popup = driver.find_element_by_css_selector(
            'button[aria-label="Dismiss" ]')
    close_popup.click()


In [214]:
def profile_connect():
    # scroll to top of profile to ensure elements can be found by webdriver
    driver.execute_script("window.scrollTo(0, 0);")
    # obtain name to personalize message
    f_name = get_first_name()
    time.sleep(2)
    # locate & click 'Connect' button located at top of profile
    driver.find_element_by_class_name('pv-s-profile-actions--connect').click()
    action_bar = driver.find_element_by_css_selector("div.artdeco-modal__actionbar")
    # locate 'Add a note' button by class
    action_bar.find_element_by_class_name('mr1').click()
    # create message using information scraped from profile
    message = f'''Hi {f_name}, I am a data scientist in the DC area. 
    I wanted to reach out, say hi, and connect with you'''.replace('\n',' ')
    time.sleep(3)
    # action to send text from message to input box
    message_input = driver.find_element_by_id('custom-message').send_keys(message)
    # send the message & connection request
    driver.find_element_by_class_name('ml1').click()

In [14]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [15]:
driver = sign_in('mn@gmail.com', 'S1!', PATH)

In [190]:
name_list = []
email_list = []
position_list = []
company_list = []
location_list = []
dates_list = []
position_list2 = []
company_list2 = []
location_list2 = []
dates_list2 = []
school_list = []
school_list2 = []

In [ ]:
loc_list = []
current_job_list = []
name_list = []
url_list = []
headline_list = []

search_df = people_scrape('data engineer', 2)

In [205]:
search_df

,name,url,current_job,location,headline,fetch
0,Vina Zhu,/in/vina-zhu-4986451a4,Data Engineer at Slalom Build,"Arlington, VA",Data Engineer,https://www.linkedin.com/in/vina-zhu-4986451a4/
1,Murali Subramanian,/in/musubr0298,Data Engineer at Searce Inc,"Austin, TX",Data Engineer at Searce Inc. | 2x GCP | 2x AWS,https://www.linkedin.com/in/musubr0298/
2,Bitthal Khaitan,/in/bitthal-khaitan-48846318,Data Engineer at CVS Health,Dallas-Fort Worth Metroplex,Data Engineer @ World’s largest Healthcare org!,https://www.linkedin.com/in/bitthal-khaitan-48...
3,Jianfang Chen,/in/jianfang-chen-2286a31b1,Data Engineering,United States,Data Engineer at Microsoft,https://www.linkedin.com/in/jianfang-chen-2286...
4,Charishma Ravoori,/in/charishmar,"Data Engineer at Lowe's Companies, Inc.","Germantown, MD",Data Engineer,https://www.linkedin.com/in/charishmar/
5,Mahdieh Taher,/in/mahdiehtaher,Data Engineer at ViacomCBS,San Francisco Bay Area,Data Engineer,https://www.linkedin.com/in/mahdiehtaher/
6,Alper Tosun,/in/alper-tosun-bba82467,Data Management Analyst at Fannie Mae - ...th...,Washington DC-Baltimore Area,Data Engineer at Fannie Mae,https://www.linkedin.com/in/alper-tosun-bba82467/
7,Alexandra Putilina,/in/putilinaa,Data Engineer Fellow at Insight Data Science,Greater Seattle Area,Data Engineer at H-E-B,https://www.linkedin.com/in/putilinaa/
8,Asfetaw Abera,/in/asfetaw-abera-54b0a2185,Data Science Fellow at The Data Incubator - C...,"Silver Spring, MD",Data Engineer at Caris Life Sciences,https://www.linkedin.com/in/asfetaw-abera-54b0...
9,Meng Zhao,/in/mzhao15,Data Engineering Fellow at Insight Data Science,"Baltimore, MD",Data Engineer at Vanguard,https://www.linkedin.com/in/mzhao15/


In [369]:
profile_urls = search_df['fetch']

In [374]:
%time
name_list = []
email_list = []
job_list = []
company_list = []
loc_list = []
date_list = []
job_list2 = []
company_list2 = []
loc_list2 = []
date_list2 = []
school_list = []

for url in profile_urls:
    driver.get(url)
    get_experience()
    get_education()
    get_email()

Wall time: 0 ns


In [377]:
detail_df = pd.DataFrame()
detail_df['job1'] = job_list
detail_df['company1'] = company_list
detail_df['location1'] = loc_list
detail_df['dates1'] = date_list
detail_df['job2'] = job_list2
detail_df['company2'] = company_list2
detail_df['location2'] = loc_list2
detail_df['dates2'] = date_list2
detail_df['school'] = school_list

In [378]:
df = pd.concat([search_df, detail_df], axis=1)

In [383]:
df['first_name'] = df['name'].map(lambda x: x.split(' ')[0])

In [384]:
df

,name,url,current_job,location,headline,fetch,job1,company1,location1,dates1,job2,company2,location2,dates2,school,first_name
0,Vina Zhu,/in/vina-zhu-4986451a4,Data Engineer at Slalom Build,"Arlington, VA",Data Engineer,https://www.linkedin.com/in/vina-zhu-4986451a4/,Data Engineer,Slalom Build Full-time,"Chicago, Illinois, United States",2021 – Present,Machine Learning Engineer,BrainCo Internship,nan,2020 – Feb 2021,The George Washington University,Vina
1,Murali Subramanian,/in/musubr0298,Data Engineer at Searce Inc,"Austin, TX",Data Engineer at Searce Inc. | 2x GCP | 2x AWS,https://www.linkedin.com/in/musubr0298/,Data Engineer,Searce Inc Full-time,"Austin, Texas, United States",2020 – Present,Wireline Engineer / PDP Automation,Schlumberger Full-time,"Midland, Texas",2018 – May 2020,The University of Texas at Austin - Red McComb...,Murali
2,Bitthal Khaitan,/in/bitthal-khaitan-48846318,Data Engineer at CVS Health,Dallas-Fort Worth Metroplex,Data Engineer @ World’s largest Healthcare org!,https://www.linkedin.com/in/bitthal-khaitan-48...,Data Engineer,CVS Health,Dallas/Fort Worth Area,2018 – Present,Information Technology Analyst,Tata Consultancy Services Full-time,Dallas-Fort Worth Metroplex,2010 – Jul 2018,WBUT,Bitthal
3,Jianfang Chen,/in/jianfang-chen-2286a31b1,Data Engineering,United States,Data Engineer at Microsoft,https://www.linkedin.com/in/jianfang-chen-2286...,Data Engineer,Microsoft,nan,2017 – Present,Business Intelligence Analyst,Amazon Full-time,nan,2017 – Apr 2017,The George Washington University,Jianfang
4,Charishma Ravoori,/in/charishmar,"Data Engineer at Lowe's Companies, Inc.","Germantown, MD",Data Engineer,https://www.linkedin.com/in/charishmar/,Data Engineer,"Lowe's Companies, Inc.","Charlotte, North Carolina, United States",2020 – Present,Data Science Engineer,Mountain State Software Solutions (MS³ ),nan,2019 – Aug 2020,University of Virginia,Charishma
5,Mahdieh Taher,/in/mahdiehtaher,Data Engineer at ViacomCBS,San Francisco Bay Area,Data Engineer,https://www.linkedin.com/in/mahdiehtaher/,Data Engineer,ViacomCBS,San Francisco,2018 – Present,Company Name\nUniversity of Minnesota - Carlso...,nan,"Minneapolis, Minnesota",2016 – Feb 2017,"University of California, Berkeley",Mahdieh
6,Alper Tosun,/in/alper-tosun-bba82467,Data Management Analyst at Fannie Mae - ...th...,Washington DC-Baltimore Area,Data Engineer at Fannie Mae,https://www.linkedin.com/in/alper-tosun-bba82467/,Company Name\nFannie Mae,nan,"Reston, Virginia, United States",2020 – Present,Assitant Manager,Papa John's International,nan,2014 – Jul 2016,James Madison University,Alper
7,Alexandra Putilina,/in/putilinaa,Data Engineer Fellow at Insight Data Science,Greater Seattle Area,Data Engineer at H-E-B,https://www.linkedin.com/in/putilinaa/,Data Engineer,H-E-B Full-time,nan,2021 – Present,Data Engineer Fellow,Insight Data Science Full-time,"Silicon Valley, California, United States",2020 – Jan 2021,Moscow Institute of Physics and Technology (St...,Alexandra
8,Asfetaw Abera,/in/asfetaw-abera-54b0a2185,Data Science Fellow at The Data Incubator - C...,"Silver Spring, MD",Data Engineer at Caris Life Sciences,https://www.linkedin.com/in/asfetaw-abera-54b0...,Data Engineer,Caris Life Sciences Full-time,"Phoenix, Arizona, United States",2021 – Present,Data Science Fellow,The Data Incubator Full-time,Online,2020 – Jun 2020,The Data Incubator,Asfetaw
9,Meng Zhao,/in/mzhao15,Data Engineering Fellow at Insight Data Science,"Baltimore, MD",Data Engineer at Vanguard,https://www.linkedin.com/in/mzhao15/,Data Engineer,Vanguard,"Malvern, Pennsylvania",2019 – Present,Company Name\nThe Johns Hopkins University,nan,"Baltimore, Maryland, United States",2019 – Jun 2019,The Johns Hopkins University,Meng


In [ ]:
driver.forward()
driver.back()

In [ ]:
driver.quit()